In [1]:
from pyspark import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import Bucketizer
from pyspark.ml.stat import Correlation

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Exploratory data analysis (EDA)

In [3]:
# Carrega dataset do desafio (basta descompactar todos na mesma pasta)
#display(dbutils.fs.ls("dbfs:/FileStore/tables/"))
df_desafio = spark.read.format("csv").options(header='true').load("/FileStore/tables/*.csv")

In [4]:
# Elimina na's e duplicados do df
df_desafio_v2 = df_desafio.dropna(how='any').dropDuplicates()

# Ajusta tipo de colunas
df_desafio_v2 = df_desafio_v2.selectExpr(
  'cast(time as timestamp) time',
  'ca',
  'unit',
  'scp',
  'station',
  'linename',
  'division',
  'desc',
  'cast(entries as int) entries',
  'cast(exits as int) exits'
)

# Features para visão temporal
df_desafio_v2 = df_desafio_v2.withColumn(
  "dt_year",
  year(col("time"))
).withColumn(
  "dt_month",
  month(col("time"))
).withColumn(
  "dt_day",
  dayofmonth(col("time"))
).withColumn(
  "dt_dayofy",
  dayofyear(col("time"))  
).withColumn(
  "dt_hour",
  hour(col("time"))
).withColumn(
  "dt_min",
  minute(col("time"))
).withColumn(
  "dt_week_no",
  weekofyear(col("time"))
).withColumn(
  "dt_int",
  unix_timestamp(col("time"))
).withColumn(
  "dt_month_year",
  date_format(col("time"), "Y-MM")
)

In [5]:
# Intervalos para buckets
splits = [-30000000, -20000000, -10000000, 0.0, 10000000, 20000000, 30000000]

# ===> Bucket: 'entries'
# dataFrame = df_desafio_v2.select(col('entries'))
# bucketizer = Bucketizer(splits=splits, inputCol="entries", outputCol="bucketedFeatures")
# bucketedData_entries = bucketizer.transform(dataFrame)
# sorted(bucketedData_entries.groupBy("bucketedFeatures").count().collect())
# [Row(bucketedFeatures=0.0, count=15721),
#  Row(bucketedFeatures=1.0, count=4028),
#  Row(bucketedFeatures=2.0, count=496),
#  Row(bucketedFeatures=4.0, count=71457425), ==> Begin
#  Row(bucketedFeatures=5.0, count=5052549),  <== End
#  Row(bucketedFeatures=6.0, count=278162),
#  Row(bucketedFeatures=7.0, count=2237294)]

# ===> Bucket: 'exits'
# dataFrame = df_desafio_v2.select(col('exits'))
# bucketizer = Bucketizer(splits=splits, inputCol="exits", outputCol="bucketedFeatures")
# bucketedData_exits = bucketizer.transform(dataFrame)
# sorted(bucketedData_exits.groupBy("bucketedFeatures").count().collect())
# [Row(bucketedFeatures=0.0, count=30576),
#  Row(bucketedFeatures=2.0, count=576),
#  Row(bucketedFeatures=3.0, count=1),
#  Row(bucketedFeatures=4.0, count=73751241), ==> Begin
#  Row(bucketedFeatures=5.0, count=3698911),  <== End
#  Row(bucketedFeatures=6.0, count=413740),
#  Row(bucketedFeatures=7.0, count=1150630)]

# Parâmetros para filtros de outliers
outlier_begin = 0
outlier_end = 20000000

df_desafio_v2 = df_desafio_v2.where((col('entries')>=outlier_begin) & (col('entries')<=outlier_end) & (col('exits')>=outlier_begin) & (col('exits')<=outlier_end))

In [6]:
# Check: 79.609.191 / 79.130.015 / 79.045.675 / 75.923.980
count_desafio = df_desafio.count()
count_desafio_na = df_desafio.dropna(how='any').count()
count_desafio_final = df_desafio.dropna(how='any').dropDuplicates().count()
count_desafio_outliers = df_desafio_v2.count()

df_amostras = sc.parallelize([
  ('antes',count_desafio,0,0,0,0),
  ('depois',0,count_desafio_final,count_desafio-count_desafio_na,count_desafio_na-count_desafio_final,count_desafio_final-count_desafio_outliers)
]).toDF(['AMOSTRAS','TOTAL','UNICO','NA','DUPLICADO','OUTLIERS'])

display(df_amostras)

In [7]:
df_amostras_v2 = sc.parallelize([
  ('',count_desafio-count_desafio_na,count_desafio_na-count_desafio_final,count_desafio_final-count_desafio_outliers)
]).toDF(['AMOSTRAS','NA','DUPLICADO','OUTLIERS'])

display(df_amostras_v2)

In [8]:
# Método para variáveis categóricas (dummys) ~14.59 minutes
lista_idx = ['ca', 'unit', 'scp', 'station', 'linename', 'division', 'desc']
indexers = [StringIndexer(inputCol=column,outputCol=column+"_idx").fit(df_desafio_v2) for column in lista_idx]
pipeline = Pipeline(stages=indexers)
df_desafio_v2 = pipeline.fit(df_desafio_v2).transform(df_desafio_v2)
#display(df_desafio_v2)

In [9]:
#75.923.980
df_desafio_v2.count()

Out[ 6 ]: 75923980

In [11]:
df_desafio_num = df_desafio_v2.select(
  'entries','exits','dt_year','dt_month','dt_day','dt_dayofy',
  'dt_hour','dt_min','dt_week_no','dt_int','dt_month_year',
  'ca_idx','unit_idx','scp_idx','station_idx','linename_idx',
  'division_idx','desc_idx'
)

#df_desafio_num.show()



+-------+-------+-------+--------+------+---------+-------+------+----------+----------+-------------+------+--------+-------+-----------+------------+------------+--------+
entries| exits|dt_year|dt_month|dt_day|dt_dayofy|dt_hour|dt_min|dt_week_no| dt_int|dt_month_year|ca_idx|unit_idx|scp_idx|station_idx|linename_idx|division_idx|desc_idx|
+-------+-------+-------+--------+------+---------+-------+------+----------+----------+-------------+------+--------+-------+-----------+------------+------------+--------+
7027076|5693669| 2010| 4| 17| 107| 4| 0| 15|1271476800| 2010-04| 89.0| 72.0| 2.0| 9.0| 70.0| 1.0| 0.0|
 508367| 400708| 2010| 4| 17| 107| 4| 0| 15|1271476800| 2010-04| 300.0| 107.0| 9.0| 100.0| 8.0| 1.0| 0.0|
4937714|2782503| 2010| 4| 17| 107| 4| 0| 15|1271476800| 2010-04| 192.0| 87.0| 26.0| 24.0| 77.0| 0.0| 0.0|
 56430| 65902| 2010| 4| 17| 107| 4| 0| 15|1271476800| 2010-04| 73.0| 16.0| 8.0| 68.0| 68.0| 0.0| 0.0|
3457669|8175192| 2010| 4| 17| 107| 4| 0| 15|1271476800| 2010-04| 25.0| 27.0| 21.0| 8.0| 18.0| 0.0| 0.0|
 885| 0| 2010| 4| 17| 107| 4| 0| 15|1271476800| 2010-04| 500.0| 105.0| 17.0| 141.0| 2.0| 0.0| 0.0|
4515607|2130293| 2010| 4| 17| 107| 4| 0| 15|1271476800| 2010-04| 107.0| 166.0| 13.0| 153.0| 2.0| 0.0| 0.0|
4194611|4761668| 2010| 4| 17| 107| 4| 0| 15|1271476800| 2010-04| 428.0| 330.0| 0.0| 236.0| 30.0| 0.0| 0.0|
2999835|2476022| 2010| 4| 17| 107| 4| 27| 15|1271478432| 2010-04| 52.0| 7.0| 45.0| 102.0| 81.0| 1.0| 12.0|
2999840|2476022| 2010| 4| 17| 107| 4| 46| 15|1271479606| 2010-04| 52.0| 7.0| 45.0| 102.0| 81.0| 1.0| 10.0|
1730926|1240796| 2010| 4| 17| 107| 5| 0| 15|1271480400| 2010-04| 226.0| 46.0| 3.0| 6.0| 1.0| 0.0| 0.0|
2352142|3554814| 2010| 4| 17| 107| 5| 0| 15|1271480400| 2010-04| 531.0| 376.0| 1.0| 441.0| 1.0| 0.0| 0.0|
 569791| 264756| 2010| 4| 17| 107| 5| 33| 15|1271482383| 2010-04| 6.0| 3.0| 3.0| 43.0| 0.0| 3.0| 0.0|
 207290| 462220| 2010| 4| 17| 107| 6| 0| 15|1271484000| 2010-04| 2.0| 0.0| 53.0| 1.0| 9.0| 0.0| 0.0|
 122731| 163935| 2010| 4| 17| 107| 7| 39| 15|1271489977| 2010-04| 80.0| 128.0| 59.0| 122.0| 0.0| 3.0| 0.0|
2999858|2476063| 2010| 4| 17| 107| 7| 51| 15|1271490689| 2010-04| 52.0| 7.0| 45.0| 102.0| 81.0| 1.0| 12.0|
2909427|3247791| 2010| 4| 17| 107| 8| 0| 15|1271491200| 2010-04| 114.0| 86.0| 27.0| 84.0| 6.0| 2.0| 0.0|
6996093|2944232| 2010| 4| 17| 107| 8| 0| 15|1271491200| 2010-04| 117.0| 83.0| 5.0| 206.0| 76.0| 2.0| 4.0|
2137973|2164060| 2010| 4| 17| 107| 8| 0| 15|1271491200| 2010-04| 444.0| 338.0| 0.0| 119.0| 50.0| 2.0| 0.0|
 60278| 21131| 2010| 4| 17| 107| 8| 0| 15|1271491200| 2010-04| 582.0| 400.0| 0.0| 355.0| 5.0| 1.0| 4.0|
+-------+-------+-------+--------+------+---------+-------+------+----------+----------+-------------+------+--------+-------+-----------+------------+------------+--------+
only showing top 20 rows

In [12]:
#df_graficos = df_desafio_v2.filter(col('dt_year')=='2017')
df_graficos = df_desafio_v2.groupBy(
  'dt_month_year',
  'dt_year',
  'dt_month',
  'dt_day',
  'dt_hour'
).agg(
  sum('entries'),
  sum('exits')
).orderBy(
  "dt_month_year"
).toPandas()

In [13]:
#display(df_graficos)